## Configuración de librerías

In [ ]:
import os
import gc
import random
from time import time
from glob import glob
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from collections import Counter
import dill as pickle

from plotly import graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots


import cv2

from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import fbeta_score, confusion_matrix

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms as T, models
from torchvision.models.resnet import ResNet18_Weights
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
#!pip install -q torchsummary --user
from torchsummary import summary

from matplotlib import pyplot as plt
%matplotlib inline

from deap import base, creator, tools, algorithms # Para el algoritmo genético

In [ ]:
# Si tenemos disponible GPU, lo usamos
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device: {device}")

In [ ]:
random.seed(101)
np.random.seed(101)
torch.manual_seed(101);

## Cargamos los datasets preprocesados.

In [ ]:
path = "../data"
path_output = "../output"

path_train = os.path.join(path, "train")
path_test = os.path.join(path, "test")
path_valid = os.path.join(path, "valid")
print(
    f"train files: {len(os.listdir(path_train))}, "
    f"test files: {len(os.listdir(path_test))}, "
    f"valid files: {len(os.listdir(path_valid))}"
)

In [ ]:
# Cargamos el dataset de train
path_train_class = os.path.join(path, "train_dataset_preprocesado.csv")
df_train = pd.read_csv(path_train_class)
print(df_train.shape)
df_train.head()

In [ ]:
# Cargamos el dataset de test
path_test_class = os.path.join(path, "test_dataset_preprocesado.csv")
df_test = pd.read_csv(path_test_class)
print(df_test.shape)
df_test.head()

In [ ]:
# Cargamos el dataset de valid
path_valid_class = os.path.join(path, "val_dataset_preprocesado.csv")
df_valid = pd.read_csv(path_valid_class)
print(df_valid.shape)
df_valid.head()

## Modelo

Para un rendimiento óptimo, resnet18 necesita una forma de entrada que sea múltiplo de 32 y en nuestro caso tenemos una entrada de tamaño 256. De 256, el múltiplo de 32 más cercano es 224.

Los valores mean y std utilizados en las transformaciones de normalización (mean=[0.485, 0.456, 0.406] y std=[0.229, 0.224, 0.225]) son estadísticas pre-calculadas sobre el conjunto de datos ImageNet. ImageNet es un gran conjunto de datos de imágenes comúnmente utilizado para entrenar modelos de visión por computadora, incluyendo redes neuronales profundas como ResNet.

Estos valores se utilizan para centrar los datos en torno a cero y escalar la varianza, lo cual puede ayudar a la red a entrenar más eficientemente. La idea es que al normalizar las imágenes con los mismos valores de mean y std con los que el modelo preentrenado fue entrenado, el rendimiento del modelo será mejor y más consistente.

In [ ]:
def obtener_transforms():
    transform_train = T.Compose([
        T.ToPILImage(),
        T.Resize(224),
        T.ToTensor(),
        T.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        )
    ])
    transform_val = T.Compose([
        T.ToPILImage(),
        T.Resize(224),
        T.ToTensor(),
        T.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        )
    ])
    return transform_train, transform_val

Definimos la clase de dataset personalizado para manipular los bathcs de datos entre RAM y disco más fácilmente. 

Algunos puntos importantes:

* `__init__`: Este es el constructor de la clase. Inicializa varias variables de instancia, incluyendo un DataFrame que contiene los datos, ohe_tags (etiquetas codificadas en one-hot), transform (una función de transformación para aplicar a las imágenes), path (la ruta o rutas a las imágenes), is_train (un booleano que indica si el conjunto de datos es para entrenamiento o prueba), y idx_tta (un índice para la técnica de aumento de test, [TTA](../referencias/TTA.md)). 

Es importante distinguir la fase de entrenamiento de la fase de prueba porque utilizamos el aumento de pruebas.
El aumento de pruebas (TTA) es útil para diversificar nuestro conjunto de datos de entrenamiento y construir un modelo más sólido. Se aplica a cada imagen para cada lote, lo que significa que no aumenta la longitud de nuestro conjunto de datos de entrenamiento, pero transforma cada imagen aleatoriamente durante el tiempo de ejecución.

* `__len__`: Este método devuelve la longitud del DataFrame, es decir, el número de elementos en el conjunto de datos.

* `__getitem__`: Este método se utiliza para obtener un elemento del conjunto de datos dado un índice. Lee la imagen correspondiente del disco, la convierte de BGR a RGB, y devuelve la imagen y su etiqueta correspondiente.

* `collate_fn`: Este método se utiliza para procesar un lote de imágenes y etiquetas. Aplica la función de transformación a cada imagen, las convierte en tensores, las permuta, y las apila en un tensor de mayor dimensión.

* `load_img`: Este método carga una imagen y su etiqueta correspondiente del conjunto de datos y las muestra en una gráfica.

* `custom_augment`: Este método aplica una serie de transformaciones a una imagen, incluyendo rotaciones y volteos. Las transformaciones son aleatorias durante el entrenamiento y no aleatorias durante las pruebas para la TTA.


In [ ]:
class YoloWasteDatasetError(Exception):
    pass

class YoloWasteDataset(Dataset):
    def __init__(self, df, ohe_tags, transform, path, is_train=True, idx_tta=None):
        super().__init__()
        self.df = df
        self.ohe_tags = ohe_tags
        self.transform = transform
        if isinstance(path, str):
            self.paths = [path]
        elif isinstance(path, (list, tuple)):
            self.paths = path
        else:
            raise YoloWasteDatasetError(f"Path type must be str, list or tuple, got: {type(path)}")
        self.is_train = is_train
        if not is_train:
            if not idx_tta in list(range(6)):
                raise YoloWasteDatasetError(
                    f"In test mode, 'idx_tta' must be an int belonging to [0, 5], got: {repr(idx_tta)}"
                )
            self.idx_tta = idx_tta

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        #filename = self.df.iloc[idx, 0] + ".jpg"  # Assuming the first column is the filename
        filename = self.df.iloc[idx, 0] # Assuming the first column is the filename
        for path in self.paths:
            if filename in os.listdir(path):
                file_path = os.path.join(path, filename)
                break
        else:
            raise YoloWasteDatasetError(f"Can't fetch {filename} among {self.paths}")
        img = cv2.imread(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = self.ohe_tags[idx]
        return img, label

    def collate_fn(self, batch):
        imgs, labels = [], []
        for (img, label) in batch:
            img = self.custom_augment(img)
            img = torch.tensor(img)
            img = img.permute(2, 0, 1)
            img = self.transform(img)
            imgs.append(img[None])
            labels.append(label)
        imgs = torch.cat(imgs).float().to(device)
        labels = torch.tensor(labels).float().to(device)
        return imgs, labels

    def load_img(self, idx, ax=None):
        img, ohe_label = self[idx]
        #label = self.df.iloc[idx].tags
        label = self.df.iloc[idx]
        title = f"{label} - {ohe_label}"
        if ax is None:
            plt.imshow(img)
            plt.title(title)
        else:
            ax.imshow(img)
            ax.set_title(title)
    
    def custom_augment(self, img):
        """
        Discrete rotation and horizontal flip.
        Random during training and non random during testing for TTA.
        Not implemented in torchvision.transforms, hence this function.
        """
        choice = np.random.randint(0, 6) if self.is_train else self.idx_tta
        if choice == 0:
            # Rotate 90
            img = cv2.rotate(img, rotateCode=cv2.ROTATE_90_CLOCKWISE)
        if choice == 1:
            # Rotate 90 and flip horizontally
            img = cv2.rotate(img, rotateCode=cv2.ROTATE_90_CLOCKWISE)
            img = cv2.flip(img, flipCode=1)
        if choice == 2:
            # Rotate 180
            img = cv2.rotate(img, rotateCode=cv2.ROTATE_180)
        if choice == 3:
            # Rotate 180 and flip horizontally
            img = cv2.rotate(img, rotateCode=cv2.ROTATE_180)
            img = cv2.flip(img, flipCode=1)
        if choice == 4:
            # Rotate 90 counter-clockwise
            img = cv2.rotate(img, rotateCode=cv2.ROTATE_90_COUNTERCLOCKWISE)
        if choice == 5:
            # Rotate 90 counter-clockwise and flip horizontally
            img = cv2.rotate(img, rotateCode=cv2.ROTATE_90_COUNTERCLOCKWISE)
            img = cv2.flip(img, flipCode=1)
        return img

In [ ]:
def get_data(df_train, df_val, df_test, path_train, batch_size):
    # Suponiendo que la primera columna es el nombre del archivo y el resto son las columnas de categoría
    category_columns = df_train.columns[1:]

    # Extraemos las etiquetas directamente del dataframe.
    ohe_tags_train = df_train[category_columns].values
    ohe_tags_val = df_val[category_columns].values
    ohe_tags_test = df_test[category_columns].values

    # Obtén las transformaciones
    transform_train, transform_val = obtener_transforms()

    # Crear datasets
    ds_train = YoloWasteDataset(df_train, ohe_tags_train, transform_train, path=path_train)
    ds_val = YoloWasteDataset(df_val, ohe_tags_val, transform_val, path=path_train)
    ds_test = YoloWasteDataset(df_test, ohe_tags_test, transform_val, path=path_train)

    # Crear dataloaders
    dl_train = DataLoader(
        ds_train,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=ds_train.collate_fn
    )
    dl_val = DataLoader(
        ds_val,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=ds_val.collate_fn
    )
    dl_test = DataLoader(
        ds_test,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=ds_test.collate_fn
    )

    return ds_train, ds_val, ds_test, dl_train, dl_val, dl_test

# Ejemplo de uso
# df_train = pd.read_csv('train.csv')
# df_val = pd.read_csv('val.csv')
# df_test = pd.read_csv('test.csv')
# path_train = 'path/to/train/images'
# ds_train, ds_val, ds_test, dl_train, dl_val, dl_test = get_data(df_train, df_val, df_test, path_train)


In [ ]:
#ds_train, ds_val, dl_train, dl_val = get_data(df_train, df_valid, path_train)
ds_train, ds_val, ds_test, dl_train, dl_val, dl_test = get_data(df_train, df_valid, df_test, path_train, batch_size=64)

In [ ]:
imgs, labels = next(iter(dl_train))
imgs.shape, labels.shape

In [ ]:
ds_train.load_img(5)

### Definición de los modelos

Descargamos pesos directamente del resnet18 previamente entrenado y congelamos todos los pesos. Sobrescribimos la última capa completamente conectada agregando dos capas densas seguidas de un sigmoide. Esta última parte del fc es la única capa a entrenar.

In [ ]:
# Resnet18
def get_resnet_model():
    weights = ResNet18_Weights.DEFAULT
    model = models.resnet18(weights=weights)
    for param in model.parameters():
        param.require_grad = False
    model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
    model.fc = nn.Sequential(
      nn.Flatten(),
      nn.Linear(512, 128), # 512 for resnet18 or 2048 for resnet 50
      nn.ReLU(inplace=True),
      nn.Dropout(.2),
      nn.Linear(128, 36),
      nn.Sigmoid()
    )
    optimizer = Adam(model.parameters(), lr=1e-4)
    loss_fn = nn.BCELoss()

    return model.to(device), optimizer, loss_fn

Ahora descargamos EfficientNet-B0 previamente entrenado y congelamos todos los pesos. Descongelamos las ultimas capas. Y modificamos la capa final completamente conectada agregando dos capas densas seguidas de un sigmoide. Esta última parte del fc es la única capa a entrenar.

In [ ]:
# EfficientNet-B0
def get_efficientnet_model():
    # Cargar el modelo EfficientNet-B0 preentrenado
    weights = models.EfficientNet_B0_Weights.DEFAULT
    model = models.efficientnet_b0(weights=weights)
    
    # Congelar los parámetros del modelo
    for param in model.parameters():
        param.requires_grad = False
    
    # Descongelar las últimas capas
    for param in model.features[-1].parameters():
        param.requires_grad = True
    
    # Modificar la capa final para que se adapte a nuestra tarea específica
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(model.classifier[1].in_features, 128), # Obtener el tamaño de entrada de la capa final
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.2),
        nn.Linear(128, 36),
        nn.Sigmoid()
    )
    
    optimizer = Adam(model.parameters(), lr=1e-2)
    loss_fn = nn.BCELoss()

    return model.to(device), optimizer, loss_fn



Ahora descargamos Vision Transformer (ViT)

In [ ]:
# Vision Transformers ViT
def get_vit_model():
    #TODO: Implementar el modelo Vision Transformer (ViT)
    pass

Ahora descargamos Swin Transformers

In [ ]:
def get_swin_transformer_model():
    #TODO: Implementar el modelo Swin Transformer
    pass

### Entrenamiento

In [ ]:
def train_batch(X, Y, model, loss_fn, optimizer):
    model.train()
    optimizer.zero_grad()
    Y_hat = model(X)
    batch_loss = loss_fn(Y_hat, Y)
    batch_loss.backward()
    optimizer.step()
    Y_hat = Y_hat.detach().float().cpu().numpy()
    
    return Y_hat, batch_loss.item()


@torch.no_grad()
def compute_val_loss(X, Y, model, loss_fn):
    model.eval()
    Y_hat = model(X)
    batch_loss = loss_fn(Y_hat, Y)
    Y_hat = Y_hat.detach().float().cpu().numpy()
    
    return Y_hat, batch_loss.item()

Elegimos entrenar nuestro modelo durante 15 épocas por defecto, mientras reducimos nuestra tasa de aprendizaje 10 veces cada 7 lotes. Monitoreamos que la pérdida de validación son nuestras métricas clave. La puntuación de validación es útil sólo como indicación secundaria, porque elegimos el umbral de clasificación de forma bastante aleatoria (0,2).
Posteriormente encontraremos el umbral más adecuado para cada objetivo.

Mas adelante efectuamos una optimización de hiperparámetros usando algoritmos genéticos. (Falta referencias)

La métrica de evaluación será f1-beta. Revisamos varias: [Métricas](../referencias/Metricas_Evaluacion.md)
 - Es una generalización del F1-score que permite ajustar el equilibrio entre Precision y Recall mediante un parámetro beta \( $\beta$ \). Un valor de \( $\beta$ > 1 \) da más peso a Recall y \( $\beta$ < 1 \) da más peso a Precision.

- \[ $\text{F1-}\beta = (1 + \beta^2) \cdot \frac{\text{Precision} \cdot \text{Recall}}{(\beta^2 \cdot \text{Precision}) + \text{Recall}}$ \]

In [ ]:
def train_model(dl_train, dl_val, idx_fold, model, optimizer, loss_fn, model_name, epochs=15):
    lr_scheduler = StepLR(optimizer, step_size=7, gamma=0.1) # Reduce el learning rate por 10 cada 7 epochs

    loss_train, loss_val = [], []
    score_train, score_val = [], []

    Y_hat_val = None
    best_loss_val = np.inf

    for idx in range(epochs):
        loss_train_epoch, loss_val_epoch = [], []
        Y_hat_train_epoch, Y_hat_val_epoch = [], []
        Y_train_epoch, Y_val_epoch = [], []

        for X, Y in tqdm(dl_train, leave=False):
            Y_hat, batch_loss = train_batch(X, Y, model, loss_fn, optimizer)
            loss_train_epoch.append(batch_loss)
            Y_hat_train_epoch.extend(Y_hat)
            Y_train_epoch.extend(Y.detach().float().cpu().numpy())

        for X, Y in tqdm(dl_val, leave=False):
            Y_hat, batch_loss = compute_val_loss(X, Y, model, loss_fn)
            loss_val_epoch.append(batch_loss)
            Y_hat_val_epoch.extend(Y_hat)
            Y_val_epoch.extend(Y.detach().float().cpu().numpy())
                
        avg_loss_train = np.mean(loss_train_epoch)
        avg_loss_val = np.mean(loss_val_epoch)

        Y_hat_train_epoch = np.array(Y_hat_train_epoch)
        Y_hat_val_epoch = np.array(Y_hat_val_epoch)
        Y_thresh_train_epoch = (Y_hat_train_epoch > .2).astype(float)
        Y_thresh_val_epoch = (Y_hat_val_epoch > .2).astype(float)
        Y_train_epoch = np.array(Y_train_epoch)
        Y_val_epoch = np.array(Y_val_epoch)
        
        score_train_epoch = fbeta_score(Y_train_epoch, Y_thresh_train_epoch, beta=2, average="samples")
        score_val_epoch = fbeta_score(Y_val_epoch, Y_thresh_val_epoch, beta=2, average="samples")
               
        # saving values for debugging
        if avg_loss_val < best_loss_val:
            best_loss_val = avg_loss_val
            Y_hat_val = Y_hat_val_epoch
            Y_thresh_val = Y_thresh_val_epoch
            Y_val = Y_val_epoch
            
        loss_train.append(avg_loss_train)
        loss_val.append(avg_loss_val)
        score_train.append(score_train_epoch)
        score_val.append(score_val_epoch)

        print(
            f"epoch: {idx}/{epochs} -- train loss: {avg_loss_train}, " \
            f"val loss: {avg_loss_val}" \
            f" -- train fbeta_score: {score_train_epoch}, " \
            f"val fbeta_score: {score_val_epoch}"
        )
        
        lr_scheduler.step()

    train_results = {
        "loss_train": loss_train,
        "loss_val": loss_val,
        "score_train": score_train,
        "score_val": score_val,
        "Y_hat_val": Y_hat_val,
        "Y_thresh_val": Y_thresh_val,
        "Y_val": Y_val,
    }
        
    
    # Guardamos el modelo
    torch.save(model, os.path.join(path_output, f"{model_name}_fold{idx_fold}.pth"))

    # Guardar los resultados del entrenamiento
    pickle.dump(train_results, open(f"../output/train_results_{model_name}_fold{idx_fold}.pkl", "wb"))
    

### Búsqueda de mejores hiperparametros

Realizamos la búsqueda de mejores hiperparámetros empleando la librería DEAP (Distributed Evolutionary Algorithms in Python).


In [ ]:
# Define el problema de optimización
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [ ]:
# Define los límites de los hiperparámetros
LIMITES = {
    "lr": (1e-5, 1e-2),
    "batch_size": (16, 128),
    "num_epochs": (10, 30),
}

In [ ]:
# Crea una función para inicializar los individuos
def init_individuo(icls, bounds):
    genome = []
    for key in bounds:
        if key == "batch_size" or key == "num_epochs":
            genome.append(random.randint(bounds[key][0], bounds[key][1]))
        else:
            genome.append(random.uniform(bounds[key][0], bounds[key][1]))
    return icls(genome)

In [ ]:
# Mapa de arquitecturas
model_map = {
    "resnet18": get_resnet_model,
    "efficientnet_b0": get_efficientnet_model,
    "vit": get_vit_model,
    "swin_transformer": get_swin_transformer_model
}

In [ ]:
# Función para evaluar los individuos
def evaluate(individual, model_name):
    lr, batch_size, num_epochs = individual

    # Asegurar que batch_size y num_epochs sean enteros
    batch_size = int(batch_size)
    num_epochs = int(num_epochs)
    
    # Preparamos los datos
    ds_train, ds_val, ds_test, dl_train, dl_val, dl_test = get_data(df_train, df_valid, df_test, path_train, batch_size)
    
    # Inicializamos el modelo y los hiperparámetros
    model_fn = model_map[model_name]
    model, optimizer, loss_fn = model_fn()
    optimizer.param_groups[0]['lr'] = lr
    
    # Entrenamos el modelo
    train_model(dl_train, dl_val, 0, model, optimizer, loss_fn, model_name, epochs=num_epochs)
    
    # Cargamos los resultados del entrenamiento
    train_results = pickle.load(open(f"../output/train_results_{model_name}_fold0.pkl", "rb"))
    score_val = train_results["score_val"]
    
    # Devolvemos el mejor score de validación (fbeta_score)
    return (max(score_val),)


In [ ]:
# Configuramos el toolbox
toolbox = base.Toolbox()
toolbox.register("individual", init_individuo, creator.Individual, LIMITES)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate, model_name="resnet18")  # Default model
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutPolynomialBounded, low=[b[0] for b in LIMITES.values()], up=[b[1] for b in LIMITES.values()], eta=1.0, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
# Ejecuta el algoritmo genético
def genetico(model_name):
    toolbox.unregister("evaluate")
    toolbox.register("evaluate", evaluate, model_name=model_name)
    
    population = toolbox.population(n=10)
    hall_of_fame = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=10, stats=stats, halloffame=hall_of_fame, verbose=True)
    
    return population, stats, hall_of_fame

In [ ]:
# Ejecutamos la optimización para cada modelo
#model_names = ["resnet18"]
model_names = ["resnet18", "efficientnet_b0", "vit", "swin_transformer"]
for model_name in model_names:
    print(f"Optimizando modelo: {model_name}")
    population, stats, hall_of_fame = genetico(model_name)
        
    # Entrenamos el mejor modelo encontrado usando el conjunto de entrenamiento completo y validación
    best_individual = hall_of_fame[0]
    lr, batch_size, num_epochs = best_individual
    
    # Aseguramos que los valores sean del tipo correcto
    lr = float(lr)
    batch_size = int(batch_size)
    num_epochs = int(num_epochs)
        
    ds_train, ds_val, ds_test, dl_train, dl_val, dl_test = get_data(df_train, df_valid, df_test, path_train, batch_size)
    model_fn = model_map[model_name]
    model, optimizer, loss_fn = model_fn()
    optimizer.param_groups[0]['lr'] = lr
        
    # Aca le colocamos al fold 1, para que no se guarde en el mismo archivo. 
    print(f"Entrenando el mejor modelo con lr={lr}, batch_size={batch_size}, num_epochs={num_epochs}")
    train_model(dl_train, dl_val, 1, model, optimizer, loss_fn, model_name, epochs=num_epochs)
        
    # Evaluación final en el conjunto de datos de test
    Y_hat_test, Y_test = [], []
    for X, Y in tqdm(dl_test, leave=False):
        Y_hat, _ = compute_val_loss(X, Y, model, loss_fn)
        Y_hat_test.extend(Y_hat)
        Y_test.extend(Y.detach().float().cpu().numpy())
        
    Y_hat_test = np.array(Y_hat_test)
    Y_test = np.array(Y_test)
    Y_thresh_test = (Y_hat_test > .2).astype(float)
        
    final_score = fbeta_score(Y_test, Y_thresh_test, beta=2, average="samples")
    print(f"Puntaje final de Fbeta en el conjunto de pruebas para {model_name}: {final_score}")

## Joss y Natanael, Faltaria:

- los gráficos, Natanael dijo que podrian ser con Tensorboard. Revisar en el código donde se efectua la optimización de hiperparámetrso, ahi se guaradn los archivos que luego pueden usarse para graficar. Tanto en la búsqueda del mejor, y una vez encontrado entrenar con ese mejor. Tambien se guarda. Para el primero se guarda con un 0 en el nombre de los archivos. Y para el entrenamiento con lo mejor con el 1 en el nombre del archivo.
Fijense que enrena con el mejor (guarda el modelo entrenado y los scores de cada epoca) y al final hace el testeo con el set de test. Eso habria que graficarlo igual.

- Referencias a la técnica de usar algoritmos genéticos para optimización de hiperparámetros

- Primero dije que podriamos emplear 4 modelos. Pero se hace pesado buscar los mejores hiperparámetros de los 4 y luego entrenerlos. Habria que dejar todo ok y dejar ejecutando todo un dia.


### Graficamos los entrenamientos


In [ ]:
# Cargamos el modelo Resnet18
model = torch.load(os.path.join(path_output, "resnet18_fold1.pth"))

# Cargamos los resultados del entrenamiento
train_results = pickle.load(open(f"../output/train_results_resnet18_fold1.pkl", "rb"))

In [ ]:
loss_train = train_results["loss_train"]
loss_val = train_results["loss_val"]
score_train = train_results["score_train"]
score_val = train_results["score_val"]

fig = make_subplots(rows=1, cols=2, subplot_titles=("Loss", "Fbeta scores"))
fig.add_trace(
    go.Scatter(
        x=list(range(len(loss_train))),
        y=loss_train,
        name="loss_train",
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=list(range(len(loss_val))),
        y=loss_val,
        name="loss_val",
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=list(range(len(score_train))),
        y=score_train,
        name="score_train",
    ),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(
        x=list(range(len(score_val))),
        y=score_val,
        name="score_val",
    ),
    row=1, col=2
)
fig.show()

In [ ]:
# Cargamos el modelo EfficientNet-B0
model = torch.load(os.path.join(path_output, "EfficientNet-B0_fold1.pth"))

# Cargamos los resultados del entrenamiento
train_results = pickle.load(open(f"../output/train_results_EfficientNet-B0_fold1.pkl", "rb"))

In [ ]:
loss_train = train_results["loss_train"]
loss_val = train_results["loss_val"]
score_train = train_results["score_train"]
score_val = train_results["score_val"]

fig = make_subplots(rows=1, cols=2, subplot_titles=("Loss", "Fbeta scores"))
fig.add_trace(
    go.Scatter(
        x=list(range(len(loss_train))),
        y=loss_train,
        name="loss_train",
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=list(range(len(loss_val))),
        y=loss_val,
        name="loss_val",
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=list(range(len(score_train))),
        y=score_train,
        name="score_train",
    ),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(
        x=list(range(len(score_val))),
        y=score_val,
        name="score_val",
    ),
    row=1, col=2
)
fig.show()